## 1. Install and Import Dependencies

If dependencies are not installed, run first:
```
pip install diffusers transformers accelerate datasets
```

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Check GPU
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")
for i in range(torch.cuda.device_count()):
    print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"    Memory: {torch.cuda.get_device_properties(i).total_memory / 1e9:.1f} GB")

## 2. Download and Load CelebA-HQ Dataset

We load the CelebA-HQ dataset first. This dataset serves as the **Target Distribution** for KSWGD and the **Ground Truth** for LDM's evaluation (FID). By loading it here, we ensure both methods are compared against the same data source.

In [ ]:
# Download CelebA-HQ to data/ folder
import subprocess
import sys
import os

try:
    from datasets import load_dataset
except ImportError:
    print("Installing datasets library...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "datasets", "-q"])
    from datasets import load_dataset

from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm

# Set data directory (alongside MNIST, CIFAR-10)
DATA_DIR = "/workspace/kswgd/data"
CELEBAHQ_CACHE = os.path.join(DATA_DIR, "CelebA-HQ")
os.makedirs(CELEBAHQ_CACHE, exist_ok=True)

print(f"Data directory: {DATA_DIR}")
print(f"CelebA-HQ cache directory: {CELEBAHQ_CACHE}")

# Download CelebA-HQ dataset to data/ folder
# Try multiple possible dataset sources
print("\nDownloading CelebA-HQ dataset...")
print("Data will be saved to data/CelebA-HQ/ folder")

dataset_sources = [
    "mattymchen/celeba-hq",           # Alternative source 1
    "datasets-community/CelebA-HQ",   # Alternative source 2
    "xinrongzhang2022/celeba-hq",     # Alternative source 3
]

celebahq_dataset = None
for source in dataset_sources:
    try:
        print(f"Trying to load: {source}")
        celebahq_dataset = load_dataset(
            source, 
            split="train",
            cache_dir=CELEBAHQ_CACHE,
            trust_remote_code=True
        )
        print(f"✓ Successfully loaded: {source}")
        break
    except Exception as e:
        print(f"✗ Failed to load: {e}")
        continue

if celebahq_dataset is None:
    raise RuntimeError("Unable to load CelebA-HQ dataset, please check network connection or download manually")

print(f"\n✓ CelebA-HQ loaded!")
print(f"  Save location: {CELEBAHQ_CACHE}")
print(f"  Total available samples: {len(celebahq_dataset)}")

## 3. Load Pretrained Unconditional Latent Diffusion Model

Using `CompVis/ldm-celebahq-256`, an **unconditional** Latent Diffusion Model (LDM) trained on CelebA-HQ.

**LDM Architecture:**
- VAE: Image ↔ Latent Space
- UNet: Denoising in Latent Space (unconditional)

In [ ]:
from diffusers import DiffusionPipeline

# Load Unconditional Latent Diffusion Model (LDM)
# This is a true unconditional model working in latent space
print("Loading Unconditional LDM (CompVis/ldm-celebahq-256)...")
print("This is an unconditional Latent Diffusion Model trained on CelebA-HQ")

ldm_pipe = DiffusionPipeline.from_pretrained("CompVis/ldm-celebahq-256")
ldm_pipe = ldm_pipe.to("cuda")

# Fix VQ-VAE scaling factor for proper image generation
# The default scaling_factor may cause color/brightness issues
ldm_pipe.vqvae.config.scaling_factor = 1.0

print("\n✓ Unconditional LDM loaded!")
print(f"  Model: CompVis/ldm-celebahq-256")
print(f"  Output size: 256×256")
print(f"  Latent Space: Yes (VAE)")
print(f"  VAE scaling factor: {ldm_pipe.vqvae.config.scaling_factor}")
print(f"  Generation: Pure unconditional, no condition input required")

## 3.1 Extract LDM's VAE

Directly use LDM's built-in VAE to maintain consistent latent space. The VAE is pre-trained as part of the LDM, so we demonstrate its reconstruction quality here before proceeding with image generation.

In [ ]:
# Directly use LDM's built-in VAE
# This way KSWGD and LDM use exactly the same latent space

print("Extracting LDM's VAE...")

# LDM pipeline contains vqvae (VQ-VAE)
vae = ldm_pipe.vqvae
vae_scaling = 1.0  # LDM's VQ-VAE doesn't need extra scaling

print(f"\n✓ VAE extracted!")
print(f"  Source: LDM's built-in VQ-VAE")
print(f"  Scaling factor: {vae_scaling}")
print(f"  Advantage: KSWGD and LDM share the same latent space")

# VAE helper functions
def _to_vae_range(x):
    """[0,1] → [-1,1]"""
    return (x * 2.0) - 1.0

def _from_vae_range(x):
    """[-1,1] → [0,1]"""
    return torch.clamp((x + 1.0) * 0.5, 0.0, 1.0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 3.2 VAE Reconstruction Quality Test

**📝 Detailed Answers to Your Questions**:

**1. Why are the images always the same?**
- **Reason**: Previous code fixed to first 5 samples (`for i in range(5)`)
- **Solution**: Changed to random sampling or selectable sampling method

**2. Where do these images come from?**
- **Source**: CelebA-HQ dataset (~30,000 high-quality face images)
- **Purpose**: Test VAE's encode→decode reconstruction capability
- **Order**: Dataset loaded in original order after loading

**3. Is the VAE pre-trained?**
- **Yes!** From `CompVis/ldm-celebahq-256` pre-trained model
- **Contains**: Complete VQ-VAE encoder + decoder
- **Training data**: Pre-trained on entire CelebA-HQ dataset

**4. What is being tested here?**
- **Purpose**: Validate VAE reconstruction quality (image → latent vector → image)
- **Importance**: VAE quality directly affects KSWGD generation performance
- **Evaluation**: Visual comparison of original vs reconstructed images

In [ ]:
# Test VAE reconstruction on real images
# 🔍 VAE Quality Test: Demonstrate LDM's built-in VQ-VAE reconstruction capability
# VAE Source: Built-in VQ-VAE from CompVis/ldm-celebahq-256 pretrained model

import random

# ============== Sampling Settings ==============
n_recon = 5
USE_RANDOM_SAMPLES = True  # True: random sampling, False: first n_recon samples
# RANDOM_SEED = 42  # Set random seed for reproducible results (comment out for pure random)
# ==============================================

print(f"=== VAE Reconstruction Quality Test ===\n")
print(f"Dataset size: {len(celebahq_dataset)} CelebA-HQ images")
print(f"Test samples: {n_recon}")
print(f"Sampling method: {'Random sampling' if USE_RANDOM_SAMPLES else 'First '+str(n_recon)+' samples'}")
print(f"VAE source: LDM pretrained model built-in VQ-VAE")
print(f"Test purpose: Validate VAE encode→decode reconstruction quality\n")

# Select test samples
if USE_RANDOM_SAMPLES:
    # Check if RANDOM_SEED is defined, if not use pure random
    if 'RANDOM_SEED' in locals():
        random.seed(RANDOM_SEED)
        print(f"Using random seed: {RANDOM_SEED}")
    else:
        print("Using pure random (no seed)")
    
    test_indices = random.sample(range(len(celebahq_dataset)), n_recon)
    print(f"Randomly selected sample indices: {test_indices}")
else:
    test_indices = list(range(n_recon))
    print(f"Fixed sample indices: {test_indices}")

fig, axes = plt.subplots(2, n_recon, figsize=(15, 6))

vae.eval()
with torch.no_grad():
    for plot_idx, sample_idx in enumerate(test_indices):
        # Original
        img = celebahq_dataset[sample_idx]["image"]
        img_tensor = T.Compose([T.Resize((256, 256)), T.ToTensor()])(img).unsqueeze(0).to(device)
        
        # Encode & Decode
        latent = vae.encode(_to_vae_range(img_tensor))
        
        if hasattr(latent, 'latents'):
            latent_code = latent.latents
        elif hasattr(latent, 'latent_dist'):
            latent_code = latent.latent_dist.mode()
        else:
            latent_code = latent[0] if isinstance(latent, tuple) else latent
            
        recon = vae.decode(latent_code).sample
        recon_img = _from_vae_range(recon).cpu().squeeze(0).permute(1, 2, 0).numpy()
        
        # Plot
        axes[0, plot_idx].imshow(np.array(img.resize((256, 256))))
        axes[0, plot_idx].set_title(f"Original #{sample_idx}", fontsize=10)
        axes[0, plot_idx].axis('off')
        
        axes[1, plot_idx].imshow(recon_img)
        axes[1, plot_idx].set_title(f"VAE Reconstructed #{sample_idx}", fontsize=10)
        axes[1, plot_idx].axis('off')

plt.suptitle("LDM Built-in VQ-VAE Reconstruction Quality Test (Encode→Decode)", fontsize=22)
plt.tight_layout()
plt.show()

print(f"\n✓ VAE reconstruction test complete!")
print(f"📊 Evaluation metric: Visual comparison of original vs reconstructed images")
print(f"🎯 Good reconstruction quality → VAE can accurately preserve and recover image information")
print(f"⚠️  Poor reconstruction quality → May affect subsequent KSWGD generation quality")
print(f"\n💡 Want to see different images?")
print(f"   - Comment out RANDOM_SEED for pure random sampling")
print(f"   - Set RANDOM_SEED to different values for reproducible results")
print(f"   - Or simply re-run this cell")

## 4. Batch Unconditional LDM Generation

In [ ]:
import time
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor

# Batch generate multiple unconditional images
n_generate = 2  # Generate 5000 images for development testing (reliable FID)
batch_size = 64    # Generate in batches (increased for 80GB GPUs)

# Check if dual-GPU is available for LDM generation
NUM_GPUS_LDM = torch.cuda.device_count()
USE_DUAL_GPU_LDM = NUM_GPUS_LDM >= 2

print(f"Detected {NUM_GPUS_LDM} GPU(s) for LDM generation")
print(f"Mode: {'Dual-GPU parallel' if USE_DUAL_GPU_LDM else 'Single-GPU'}")
print(f"Generating {n_generate} unconditional LDM images (batch_size={batch_size})...")
print(f"This will take a while...")

start_time = time.time()

if USE_DUAL_GPU_LDM:
    # ============== Dual-GPU LDM Generation ==============
    from diffusers import DiffusionPipeline
    
    # Calculate split: each GPU generates half
    n_gpu0 = n_generate // 2
    n_gpu1 = n_generate - n_gpu0
    
    print(f"  GPU 0: {n_gpu0} images")
    print(f"  GPU 1: {n_gpu1} images")
    
    # Load LDM on GPU 1 (GPU 0 already has ldm_pipe)
    print("Loading LDM on GPU 1...")
    ldm_pipe_gpu1 = DiffusionPipeline.from_pretrained("CompVis/ldm-celebahq-256")
    ldm_pipe_gpu1 = ldm_pipe_gpu1.to("cuda:1")
    ldm_pipe_gpu1.vqvae.config.scaling_factor = 1.0
    ldm_pipe_gpu1.set_progress_bar_config(disable=True)
    
    # Also disable progress bar for GPU 0
    ldm_pipe.set_progress_bar_config(disable=True)
    
    def generate_on_gpu(pipe, n_images, batch_size, gpu_id):
        """Generate images on a specific GPU"""
        images = []
        n_batches = (n_images + batch_size - 1) // batch_size
        
        for batch_idx in tqdm(range(n_batches), desc=f"LDM (GPU {gpu_id})"):
            current_batch_size = min(batch_size, n_images - batch_idx * batch_size)
            result = pipe(
                batch_size=current_batch_size,
                num_inference_steps=500,
            )
            images.extend(result.images)
            
            # Clear cache periodically
            if (batch_idx + 1) % 20 == 0:
                torch.cuda.empty_cache()
        
        return images
    
    # Run both GPUs in parallel using ThreadPoolExecutor
    with ThreadPoolExecutor(max_workers=2) as executor:
        future_gpu0 = executor.submit(generate_on_gpu, ldm_pipe, n_gpu0, batch_size, 0)
        future_gpu1 = executor.submit(generate_on_gpu, ldm_pipe_gpu1, n_gpu1, batch_size, 1)
        
        images_gpu0 = future_gpu0.result()
        images_gpu1 = future_gpu1.result()
    
    # Combine results
    ldm_generated_images = images_gpu0 + images_gpu1
    
    # Clean up GPU 1 pipeline to free memory
    del ldm_pipe_gpu1
    torch.cuda.empty_cache()
    
else:
    # ============== Single-GPU LDM Generation ==============
    ldm_pipe.set_progress_bar_config(disable=True)
    
    ldm_generated_images = []
    n_batches = (n_generate + batch_size - 1) // batch_size

    for batch_idx in tqdm(range(n_batches), desc="LDM Generation"):
        current_batch_size = min(batch_size, n_generate - batch_idx * batch_size)
        result = ldm_pipe(
            batch_size=current_batch_size,
            num_inference_steps=500,
        )
        ldm_generated_images.extend(result.images)
        
        # Clear cache periodically
        if (batch_idx + 1) % 50 == 0:
            torch.cuda.empty_cache()

elapsed = time.time() - start_time
print(f"\n✓ LDM generation complete!")
print(f"Total images: {len(ldm_generated_images)}")
print(f"Total time: {elapsed:.1f} seconds ({elapsed/60:.1f} minutes)")
print(f"Average per image: {elapsed/len(ldm_generated_images):.2f} seconds")
if USE_DUAL_GPU_LDM:
    print(f"Speedup: ~{2.0:.1f}x (dual-GPU parallel)")

# Visualization (show first 16)
fig, axes = plt.subplots(4, 4, figsize=(10, 10))
axes = axes.flatten()

for idx in range(min(16, len(ldm_generated_images))):
    axes[idx].imshow(ldm_generated_images[idx])
    # axes[idx].set_title(f"#{idx+1}")
    axes[idx].axis('off')

# Get image size from generated images
_ldm_img_size = ldm_generated_images[0].size[0] if hasattr(ldm_generated_images[0], 'size') else 256
plt.suptitle(f"LDM Generated Images (CelebA-HQ {_ldm_img_size}x{_ldm_img_size})", fontsize=22, y=0.98)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

## 4.1 Super-Resolution (Real-ESRGAN)

The native output of LDM is 256x256. To match the visual quality of high-resolution papers, we apply **Real-ESRGAN** super-resolution to upscale images to 1024x1024.

This is **Method A** - the same approach used in many diffusion model papers for high-quality output.

In [ ]:
# Real-ESRGAN + GFPGAN Super-Resolution Setup (Dual-GPU Support)
import os
import cv2
from concurrent.futures import ThreadPoolExecutor
from basicsr.archs.rrdbnet_arch import RRDBNet
from realesrgan import RealESRGANer
from gfpgan import GFPGANer

# Create figures directory if not exists
os.makedirs('/workspace/kswgd/figures', exist_ok=True)

# Check number of GPUs
NUM_GPUS = torch.cuda.device_count()
USE_DUAL_GPU = NUM_GPUS >= 2
print(f"Detected {NUM_GPUS} GPU(s), using {'Dual-GPU' if USE_DUAL_GPU else 'Single-GPU'} mode for upscaling")

# Initialize Real-ESRGAN and GFPGAN for each GPU
model_path = '/workspace/kswgd/weights/RealESRGAN_x4plus.pth'
gfpgan_path = '/workspace/kswgd/weights/GFPGANv1.3.pth'

def create_upscaler(gpu_id):
    """Create Real-ESRGAN + GFPGAN instances on specified GPU"""
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, num_block=23, num_grow_ch=32, scale=4)
    upsampler = RealESRGANer(
        scale=4,
        model_path=model_path,
        model=model,
        tile=0,
        tile_pad=10,
        pre_pad=0,
        half=True,
        gpu_id=gpu_id
    )
    face_enhancer = GFPGANer(
        model_path=gfpgan_path,
        upscale=4,
        arch='clean',
        channel_multiplier=2,
        bg_upsampler=upsampler
    )
    return upsampler, face_enhancer

print("Initializing Real-ESRGAN + GFPGAN...")
if USE_DUAL_GPU:
    # Create instances on both GPUs
    upsampler_0, face_enhancer_0 = create_upscaler(0)
    upsampler_1, face_enhancer_1 = create_upscaler(1)
    upsamplers = [upsampler_0, upsampler_1]
    face_enhancers = [face_enhancer_0, face_enhancer_1]
    print(f"  ✓ GPU 0: Real-ESRGAN + GFPGAN initialized")
    print(f"  ✓ GPU 1: Real-ESRGAN + GFPGAN initialized")
else:
    # Single GPU mode
    upsampler_0, face_enhancer_0 = create_upscaler(0)
    upsamplers = [upsampler_0]
    face_enhancers = [face_enhancer_0]
    print(f"  ✓ GPU 0: Real-ESRGAN + GFPGAN initialized")

def preprocess_image(img_bgr, use_gaussian=True, use_bilateral=True, use_color_norm=True):
    """
    Apply preprocessing before GFPGAN:
    1. Gaussian blur - reduce noise
    2. Bilateral filter - edge-preserving smoothing
    3. Color normalization - adjust color distribution (mean normalization)
    """
    processed = img_bgr.copy().astype(np.float32)
    
    # 1. Gaussian Blur (slight smoothing, small kernel size)
    if use_gaussian:
        processed = cv2.GaussianBlur(processed, (3, 3), sigmaX=0.5)
    
    # 2. Bilateral Filter (edge-preserving smoothing)
    if use_bilateral:
        processed = cv2.bilateralFilter(processed.astype(np.uint8), d=5, sigmaColor=30, sigmaSpace=30).astype(np.float32)
    
    # 3. Color Normalization (color space mean normalization)
    if use_color_norm:
        mean_b = np.mean(processed[:, :, 0])
        mean_g = np.mean(processed[:, :, 1])
        mean_r = np.mean(processed[:, :, 2])
        target_mean = 127.5
        alpha = 0.3
        processed[:, :, 0] = processed[:, :, 0] + alpha * (target_mean - mean_b)
        processed[:, :, 1] = processed[:, :, 1] + alpha * (target_mean - mean_g)
        processed[:, :, 2] = processed[:, :, 2] + alpha * (target_mean - mean_r)
        processed = np.clip(processed, 0, 255)
    
    return processed.astype(np.uint8)

def process_single_image(args):
    """Process a single image with specified GPU"""
    img, gpu_id, use_face_enhance, use_preprocess = args
    face_enhancer = face_enhancers[gpu_id]
    upsampler = upsamplers[gpu_id]
    
    if isinstance(img, Image.Image):
        img_np = np.array(img)
    else:
        if img.ndim == 3 and img.shape[0] == 3:
            img_np = np.transpose(img, (1, 2, 0))
        else:
            img_np = img
        if img_np.max() <= 1.0:
            img_np = (img_np * 255).astype(np.uint8)
    
    img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    
    if use_preprocess:
        img_bgr = preprocess_image(img_bgr, use_gaussian=True, use_bilateral=True, use_color_norm=True)
    
    if use_face_enhance:
        _, _, output_bgr = face_enhancer.enhance(img_bgr, has_aligned=False, only_center_face=False, paste_back=True)
    else:
        output_bgr, _ = upsampler.enhance(img_bgr, outscale=4)
    
    output_rgb = cv2.cvtColor(output_bgr, cv2.COLOR_BGR2RGB)
    return output_rgb

def upscale_images(images_list, use_face_enhance=True, use_preprocess=True, desc="Upscaling"):
    """
    Upscale images using GFPGAN (face enhancement) or Real-ESRGAN (basic upscale).
    Automatically uses dual-GPU parallel processing if available.
    """
    n_images = len(images_list)
    
    if USE_DUAL_GPU and n_images >= 2:
        # Dual-GPU parallel processing
        print(f"  Using Dual-GPU parallel processing...")
        mid = n_images // 2
        
        # Assign images to GPUs: first half to GPU 0, second half to GPU 1
        args_gpu0 = [(img, 0, use_face_enhance, use_preprocess) for img in images_list[:mid]]
        args_gpu1 = [(img, 1, use_face_enhance, use_preprocess) for img in images_list[mid:]]
        
        results_gpu0 = []
        results_gpu1 = []
        
        # Use ThreadPoolExecutor to run both GPUs in parallel
        with ThreadPoolExecutor(max_workers=2) as executor:
            # Submit GPU 0 batch
            future_gpu0 = executor.submit(
                lambda args_list: [process_single_image(args) for args in tqdm(args_list, desc=f"{desc} (GPU 0)")],
                args_gpu0
            )
            # Submit GPU 1 batch  
            future_gpu1 = executor.submit(
                lambda args_list: [process_single_image(args) for args in tqdm(args_list, desc=f"{desc} (GPU 1)")],
                args_gpu1
            )
            
            results_gpu0 = future_gpu0.result()
            results_gpu1 = future_gpu1.result()
        
        # Combine results in original order
        upscaled_images = results_gpu0 + results_gpu1
    else:
        # Single-GPU processing
        upscaled_images = []
        for img in tqdm(images_list, desc=desc):
            result = process_single_image((img, 0, use_face_enhance, use_preprocess))
            upscaled_images.append(result)
    
    return upscaled_images

print("✓ Upscalers initialized!")
print(f"  - Real-ESRGAN: 4x basic upscale")
print(f"  - GFPGAN: 4x upscale + face enhancement + color correction")
print(f"  - Preprocessing: Gaussian blur + Bilateral filter + Color normalization")
print(f"  - Mode: {'Dual-GPU parallel' if USE_DUAL_GPU else 'Single-GPU sequential'}")

# ============== Upscale LDM Generated Images ==============
print("\n" + "=" * 60)
print("Upscaling LDM Generated Images with GFPGAN")
print("(with preprocessing: Gaussian + Bilateral + Color Norm)")
print("=" * 60)

# Use all generated images from previous cell
n_test = len(ldm_generated_images)
print(f"\nEnhancing {n_test} LDM images with preprocessing + GFPGAN...")
ldm_enhanced = upscale_images(ldm_generated_images, use_face_enhance=True, use_preprocess=True, desc="Preprocess+GFPGAN")

# Visualization (show up to 16 in a grid)
n_show = min(16, n_test)
n_cols = 4
n_rows = (n_show + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 4 * n_rows))
axes = np.asarray(axes).reshape(-1)

for i in range(n_show):
    axes[i].imshow(ldm_enhanced[i])
    # axes[i].set_title(f"#{i+1}")
    axes[i].axis('off')

for i in range(n_show, len(axes)):
    axes[i].axis('off')

# Get image size from enhanced images
_ldm_enhanced_size = ldm_enhanced[0].shape[1] if isinstance(ldm_enhanced[0], np.ndarray) else ldm_enhanced[0].size[0]
plt.suptitle(f"LDM Enhanced Images (CelebA-HQ {_ldm_enhanced_size}x{_ldm_enhanced_size})", fontsize=22, y=0.98)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.savefig('/workspace/kswgd/figures/ldm_gfpgan_enhanced.png', dpi=150, bbox_inches='tight')
plt.show()

# Store enhanced images for later use
ldm_upscaled = ldm_enhanced

print(f"\n✓ Enhancement complete!")
print(f"  Original LDM: 256x256")
print(f"  Enhanced: {ldm_upscaled[0].shape[1]}x{ldm_upscaled[0].shape[0]}")
print(f"  Preprocessing applied: Gaussian blur (3x3, σ=0.5) + Bilateral (d=5) + Color norm (α=0.3)")

In [ ]:
# Section 6.5: Memory Cleanup
# To avoid OOM during KSWGD, we release the LDM UNet and other components that are no longer needed.
import gc

print(f"GPU Memory before cleanup: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

if 'ldm_pipe' in globals():
    # We only need the VAE for KSWGD, so we can release the UNet (the largest part)
    # This can free up 2-4 GB of VRAM.
    print("Releasing LDM UNet and other components...")
    if hasattr(ldm_pipe, 'unet'):
        ldm_pipe.unet = None
    if hasattr(ldm_pipe, 'text_encoder'):
        ldm_pipe.text_encoder = None
    if hasattr(ldm_pipe, 'scheduler'):
        ldm_pipe.scheduler = None
    
gc.collect()
torch.cuda.empty_cache()

print(f"GPU Memory after cleanup: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

## 7. KSWGD Unconditional Generation (Core Experiment)

Using KSWGD (Koopman Spectral Wasserstein Gradient Descent) to perform **unconditional** particle transport generation in latent space.

**Unconditional Pipeline:**
```
Training Data (CelebA-HQ) → VAE Encoder → Latent Z_tar (Target Distribution Samples)
                                              ↓
Random Noise N(0,1) → KSWGD Transport → Z_gen (Unconditional Generation)
                                              ↓
                                   VAE Decoder → Generated Image
```

**Comparison of Two Unconditional Methods (Fair Comparison, Same Dataset):**
| | LDM (Section 2-4) | KSWGD (Section 7) |
|---|---|---|
| Dataset | CelebA-HQ | CelebA-HQ |
| Generation Process | Random Noise → Denoising | Random Noise → KSWGD |
| Core Component | UNet | Kernel Matrix + Eigendecomposition |
| Condition Input | None | None |
| Iteration Steps | 200 | Adjustable (e.g., 300) |

In [ ]:
# Import KSWGD required libraries and custom kernel functions
from sklearn.metrics import pairwise_distances
from tqdm.auto import trange

# Import your kernel functions
from grad_ker1 import grad_ker1
from K_tar_eval import K_tar_eval

# Try to import GPU version
try:
    import cupy as cp
    from grad_ker1_gpu import grad_ker1 as grad_ker1_gpu
    from K_tar_eval_gpu import K_tar_eval as K_tar_eval_gpu
    GPU_KSWGD = True
    print("✓ GPU KSWGD backend available (CuPy)")
except Exception as e:
    cp = None
    grad_ker1_gpu = None
    K_tar_eval_gpu = None
    GPU_KSWGD = False
    print(f"✗ GPU KSWGD backend not available: {e}")
    print("  Using CPU backend instead")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

In [ ]:
# Encode CelebA-HQ images and apply PCA dimensionality reduction
from sklearn.decomposition import IncrementalPCA
from torchvision import transforms as T
from tqdm.auto import tqdm
import pickle
import os

# Image preprocessing
transform_celebahq = T.Compose([
    T.Resize((256, 256)),
    T.ToTensor(),
])

# ============== PCA Settings ==============
# Set USE_PCA = False to use full 1024-dim latent space (better quality, slower)
# Set USE_PCA = True to reduce to REDUCED_DIM dimensions (faster, may lose detail)
USE_PCA = True  # <-- Toggle this to switch between PCA and full latent space
REDUCED_DIM = 64  # Only used if USE_PCA = True
# ==========================================

# ============== Caching Settings ==============
# Set LOAD_FROM_CACHE = True to load pre-computed VAE encodings from cache
# Set LOAD_FROM_CACHE = False to recompute (takes ~10 minutes)
LOAD_FROM_CACHE = True
CACHE_DIR = "/workspace/kswgd/cache"
os.makedirs(CACHE_DIR, exist_ok=True)
# ==============================================

# Define GPUPCAResult class BEFORE loading cache (required for pickle deserialization)
class GPUPCAResult:
    """Custom PCA result class for GPU-accelerated PCA inverse transform"""
    def __init__(self, components, mean, singular_values, explained_variance_ratio):
        self.components_ = components.T  # (REDUCED_DIM, 1024)
        self.mean_ = mean.squeeze()  # (1024,)
        self.singular_values_ = singular_values
        self.explained_variance_ratio_ = explained_variance_ratio
    
    def inverse_transform(self, X):
        # X: (N, REDUCED_DIM) -> (N, 1024)
        return X @ self.components_ + self.mean_

max_samples = 28000

# Generate cache filename based on settings
pca_suffix = f"pca{REDUCED_DIM}" if USE_PCA else "pca_off"
cache_path = os.path.join(CACHE_DIR, f"vae_encoding_n{max_samples}_{pca_suffix}.pkl")

# Try to load from cache
if LOAD_FROM_CACHE and os.path.exists(cache_path):
    print(f"Loading VAE encodings from cache: {cache_path}")
    with open(cache_path, 'rb') as f:
        cache_data = pickle.load(f)
    Z_all = cache_data['Z_all']
    latent_dim = cache_data['latent_dim']
    pca = cache_data['pca']
    full_latent_shape = cache_data['full_latent_shape']
    print(f"✓ Loaded from cache!")
    print(f"  Z_all shape: {Z_all.shape}")
    print(f"  Latent dim: {latent_dim}")
    print(f"  PCA: {'Enabled' if pca is not None else 'Disabled'}")
else:
    print(f"Cache not found or disabled. Computing VAE encodings...")
    
    all_latents = []
    vae.eval()
    print(f"Encoding {max_samples} images to latent space...")

    with torch.no_grad():
        for i in tqdm(range(min(max_samples, len(celebahq_dataset))), desc="Encoding"):
            img = celebahq_dataset[i]["image"]
            img_tensor = transform_celebahq(img).unsqueeze(0).to(device)
            
            # Encode to latent
            latent = vae.encode(_to_vae_range(img_tensor))
            latent_code = latent.latents if hasattr(latent, 'latents') else (latent.latent_dist.mode() if hasattr(latent, 'latent_dist') else latent[0])
            latent_code = latent_code * vae_scaling # (1, 4, 16, 16)
            
            all_latents.append(latent_code.view(1, -1).cpu().numpy())

    Z_flat = np.concatenate(all_latents, axis=0)
    print(f"Latent vectors collected: {Z_flat.shape}")

    # Apply PCA or use full latent space
    if USE_PCA:
        import time as _time
        print(f"Applying GPU-accelerated PCA (1024 -> {REDUCED_DIM})...")
        _pca_start = _time.time()
        
        # GPU PCA using PyTorch's pca_lowrank (much faster than sklearn CPU)
        Z_flat_tensor = torch.from_numpy(Z_flat).float().to(device)
        
        # Center the data (required for PCA)
        Z_mean_pca = Z_flat_tensor.mean(dim=0, keepdim=True)
        Z_centered = Z_flat_tensor - Z_mean_pca
        
        # Compute PCA using randomized SVD (GPU-accelerated)
        # q parameter controls oversampling for better accuracy
        U, S, V = torch.pca_lowrank(Z_centered, q=REDUCED_DIM, niter=2)
        
        # Project to reduced space: Z_reduced = Z_centered @ V
        Z_reduced_tensor = Z_centered @ V
        Z_all = Z_reduced_tensor.cpu().numpy()
        
        # Store PCA components for inverse transform
        pca_components = V.cpu().numpy()  # (1024, REDUCED_DIM)
        pca_mean = Z_mean_pca.cpu().numpy()  # (1, 1024)
        pca_singular_values = S.cpu().numpy()
        
        # Calculate explained variance ratio
        total_var = torch.var(Z_flat_tensor, dim=0).sum().item()
        explained_var = (S ** 2).cpu().numpy() / (Z_flat.shape[0] - 1)
        explained_var_ratio = explained_var / (total_var + 1e-8)
        
        # Use the GPUPCAResult class defined above
        pca = GPUPCAResult(pca_components, pca_mean, pca_singular_values, explained_var_ratio)
        latent_dim = Z_all.shape[1]
        
        _pca_elapsed = _time.time() - _pca_start
        print(f"\n✓ GPU PCA complete! Time: {_pca_elapsed:.2f}s")
        print(f"  Backend: PyTorch GPU (torch.pca_lowrank)")
        print(f"  Original latent shape: {Z_flat.shape[1]}")
        print(f"  Reduced latent shape:  {Z_all.shape[1]}")
        print(f"  Explained variance ratio: {np.sum(explained_var_ratio):.4f}")
        
        # Clean up GPU memory
        del Z_flat_tensor, Z_centered, U, S, V, Z_reduced_tensor, Z_mean_pca
        torch.cuda.empty_cache()
    else:
        print(f"\n✓ Using FULL 1024-dim latent space (no PCA)")
        Z_all = Z_flat  # Use full 1024-dim latent directly
        latent_dim = Z_all.shape[1]
        pca = None  # No PCA object
        print(f"  Latent dimension: {latent_dim}")

    # Record shapes for decoding
    with torch.no_grad():
        dummy = torch.zeros(1, 3, 256, 256, device=device)
        dummy_latent = vae.encode(_to_vae_range(dummy))
        dummy_code = dummy_latent.latents if hasattr(dummy_latent, 'latents') else (dummy_latent.latent_dist.mode() if hasattr(dummy_latent, 'latent_dist') else dummy_latent[0])
        full_latent_shape = dummy_code.shape[1:] # (4, 16, 16)

    # Save to cache
    print(f"\nSaving to cache: {cache_path}")
    cache_data = {
        'Z_all': Z_all,
        'latent_dim': latent_dim,
        'pca': pca,
        'full_latent_shape': full_latent_shape,
        'USE_PCA': USE_PCA,
        'REDUCED_DIM': REDUCED_DIM if USE_PCA else None,
        'max_samples': max_samples
    }
    with open(cache_path, 'wb') as f:
        pickle.dump(cache_data, f)
    print(f"✓ Cache saved!")

print(f"Full Latent Shape: {full_latent_shape}")

### 7.1 Latent Space Dimensionality Reduction: PCA

We use **PCA (Principal Component Analysis)** to reduce the dimensionality of the VQ-VAE latent space from 1024 (4x16x16) to 128.

**Why PCA?**
1.  **Efficiency**: KSWGD's kernel matrix and eigendecomposition scale with the dimensionality of the particles. Reducing to 128 dimensions balances detail preservation and transport speed.
2.  **Noise Reduction**: PCA captures the most significant variations in the latent space, effectively filtering out high-frequency noise.
3.  **Reversibility**: We can use `pca.inverse_transform` to project the generated particles back into the 1024-dimensional space before decoding them with the VAE.

In [ ]:
# Standardize latent codes and build KSWGD kernel operator
# Standardization
Z_mean = np.mean(Z_all, axis=0, keepdims=True)
Z_std = np.std(Z_all, axis=0, keepdims=True) + 1e-8
Z_std = Z_std.astype(np.float64)
Z_mean = Z_mean.astype(np.float64)
X_tar = ((Z_all - Z_mean) / Z_std).astype(np.float64)

print(f"After standardization: mean={X_tar.mean():.4f}, std={X_tar.std():.4f}")

# Compute squared sum of target samples (for kernel function)
sq_tar = np.sum(X_tar ** 2, axis=1)

# Compute pairwise distances and bandwidth epsilon
dists = pairwise_distances(X_tar, metric="euclidean")
eps_kswgd = np.median(dists**2) / (2.0 * np.log(X_tar.shape[0] + 1))
eps_kswgd = float(max(eps_kswgd, 1e-6))

print(f"KSWGD epsilon: {eps_kswgd:.6f}")
print(f"Distance stats: min={dists[dists>0].min():.4f}, median={np.median(dists):.4f}, max={dists.max():.4f}")

# Build data kernel matrix
data_kernel = np.exp(-dists**2 / (2.0 * eps_kswgd))

# Normalization
p_x = np.sqrt(np.sum(data_kernel, axis=1))
data_kernel_norm = data_kernel / (p_x[:, None] * p_x[None, :] + 1e-12)
D_y = np.sum(data_kernel_norm, axis=0)
rw_kernel = 0.5 * (data_kernel_norm / (D_y + 1e-12) + data_kernel_norm / (D_y[:, None] + 1e-12))
rw_kernel = np.nan_to_num(rw_kernel)

print(f"Kernel matrix built, shape: {rw_kernel.shape}")

# latent_dim is already set in the previous cell (Spatial Mean Pooling)
print(f"KSWGD latent_dim: {latent_dim}")

In [ ]:
# Compute eigendecomposition and KSWGD weights
import time
import threading
from scipy.sparse.linalg import eigsh

print("Computing eigendecomposition...")
start_time = time.time()

# ============== Truncated Eigendecomposition Settings ==============
# k: number of largest eigenvalues to compute
# Using truncated decomposition is much faster for large matrices (e.g., 30000x30000)
k_eig = 300  # Default: compute top 300 eigenvalues (truncated)
# k_eig = rw_kernel.shape[0]  # Uncomment this line to compute ALL eigenvalues (full decomposition)
USE_GPU_EIGSH = True  # Use GPU for truncated eigendecomposition (torch.lobpcg)
# ===================================================================

n_samples = rw_kernel.shape[0]
use_truncated = k_eig < n_samples

# Estimate time based on matrix size
if use_truncated:
    est_time = n_samples * k_eig / 1e7  # Rough estimate for truncated
else:
    est_time = (n_samples ** 2.5) / 1e9  # O(n^3) but optimized

# Progress tracking with elapsed time display
_eig_done = threading.Event()
_eig_result = {}

def _show_progress():
    """Show elapsed time while eigendecomposition is running"""
    import sys
    symbols = ['⠋', '⠙', '⠹', '⠸', '⠼', '⠴', '⠦', '⠧', '⠇', '⠏']
    idx = 0
    while not _eig_done.is_set():
        elapsed = time.time() - start_time
        sys.stdout.write(f"\r  {symbols[idx]} Eigendecomposition in progress... {elapsed:.1f}s elapsed")
        sys.stdout.flush()
        idx = (idx + 1) % len(symbols)
        _eig_done.wait(0.2)
    sys.stdout.write("\r" + " " * 60 + "\r")  # Clear line
    sys.stdout.flush()

def _run_eigsh_cpu():
    """Run truncated eigendecomposition on CPU (scipy)"""
    lambda_ns_partial, phi_partial = eigsh(rw_kernel, k=k_eig, which='LM')
    _eig_result['lambda'] = lambda_ns_partial
    _eig_result['phi'] = phi_partial
    _eig_done.set()

def _run_eigsh_gpu():
    """Run truncated eigendecomposition on GPU (torch.lobpcg)"""
    # torch.lobpcg computes largest eigenvalues of symmetric positive definite matrices
    rw_kernel_torch = torch.from_numpy(rw_kernel).float().to(device)
    
    # Random initial vectors for lobpcg
    X0 = torch.randn(n_samples, k_eig, device=device, dtype=torch.float32)
    
    # lobpcg for largest eigenvalues (works well for positive semi-definite matrices)
    try:
        eigenvalues, eigenvectors = torch.lobpcg(rw_kernel_torch, k=k_eig, X=X0, largest=True, niter=100)
        _eig_result['lambda'] = eigenvalues.cpu().numpy()
        _eig_result['phi'] = eigenvectors.cpu().numpy()
    except Exception as e:
        print(f"\n  Warning: GPU lobpcg failed ({e}), falling back to CPU...")
        _eig_done.set()
        return False
    
    del rw_kernel_torch, X0, eigenvalues, eigenvectors
    torch.cuda.empty_cache()
    _eig_done.set()
    return True

def _run_eigh_gpu():
    """Run full eigendecomposition on GPU"""
    rw_kernel_torch = torch.from_numpy(rw_kernel).to(device)
    lambda_ns_torch, phi_torch = torch.linalg.eigh(rw_kernel_torch)
    _eig_result['lambda'] = lambda_ns_torch.cpu().numpy()[::-1].copy()
    _eig_result['phi'] = phi_torch.cpu().numpy()[:, ::-1].copy()
    del rw_kernel_torch, lambda_ns_torch, phi_torch
    torch.cuda.empty_cache()
    _eig_done.set()

def _run_eigh_cpu():
    """Run full eigendecomposition on CPU"""
    lambda_ns_raw, phi_raw = np.linalg.eigh(rw_kernel)
    _eig_result['lambda'] = lambda_ns_raw[::-1]
    _eig_result['phi'] = phi_raw[:, ::-1]
    _eig_done.set()

if use_truncated:
    print(f"  Using TRUNCATED eigendecomposition (top {k_eig} of {n_samples})...")
    
    # Try GPU first if enabled
    use_gpu_eig = USE_GPU_EIGSH and torch.cuda.is_available()
    
    if use_gpu_eig:
        print(f"  Backend: PyTorch GPU (torch.lobpcg)")
        print(f"  Estimated time: ~{est_time/5:.0f}-{est_time:.0f} seconds (GPU accelerated)")
        
        # Start progress display thread
        progress_thread = threading.Thread(target=_show_progress, daemon=True)
        progress_thread.start()
        
        # Try GPU lobpcg
        gpu_success = _run_eigsh_gpu()
        progress_thread.join(timeout=1)
        
        if not gpu_success or 'lambda' not in _eig_result:
            # Fallback to CPU
            print(f"  Falling back to CPU (scipy.sparse.linalg.eigsh)...")
            _eig_done.clear()
            _eig_result.clear()
            start_time = time.time()  # Reset timer
            progress_thread = threading.Thread(target=_show_progress, daemon=True)
            progress_thread.start()
            _run_eigsh_cpu()
            progress_thread.join(timeout=1)
    else:
        print(f"  Backend: CPU (scipy.sparse.linalg.eigsh)")
        print(f"  Estimated time: ~{est_time:.0f}-{est_time*2:.0f} seconds")
        
        # Start progress display thread
        progress_thread = threading.Thread(target=_show_progress, daemon=True)
        progress_thread.start()
        
        # Run CPU eigendecomposition
        _run_eigsh_cpu()
        progress_thread.join(timeout=1)
    
    # Sort in descending order
    sort_idx = np.argsort(_eig_result['lambda'])[::-1]
    lambda_ns = _eig_result['lambda'][sort_idx]
    phi = _eig_result['phi'][:, sort_idx]
else:
    print(f"  Using FULL eigendecomposition ({n_samples} x {n_samples})...")
    print(f"  Estimated time: ~{est_time:.0f}-{est_time*3:.0f} seconds")
    
    # Start progress display thread
    progress_thread = threading.Thread(target=_show_progress, daemon=True)
    progress_thread.start()
    
    if torch.cuda.is_available():
        print("    Using GPU acceleration...")
        _run_eigh_gpu()
    else:
        print("    Using CPU...")
        _run_eigh_cpu()
    
    progress_thread.join(timeout=1)
    lambda_ns = _eig_result['lambda']
    phi = _eig_result['phi']

elapsed = time.time() - start_time
print(f"✓ Eigendecomposition complete! Time: {elapsed:.1f}s")

# Set regularization parameters
tol = 1e-6
reg = 1e-3
latent_dim = X_tar.shape[1]

# Compute inverse eigenvalues (fix negative number issue)
lambda_ = lambda_ns - 1.0
inv_lambda = np.zeros_like(lambda_)
# Only compute inverse for positive lambda_, set negative to 0
positive_mask = lambda_[1:] > tol
inv_lambda[1:][positive_mask] = 1.0 / (np.abs(lambda_[1:][positive_mask]) + reg)
inv_lambda *= eps_kswgd

# Count eigenvalues above tolerance (always compute this regardless of truncation)
above_tol = int(np.sum(lambda_ns >= tol))

# Truncate small eigenvalues
lambda_ns_inv = np.zeros_like(lambda_ns)
mask = lambda_ns >= tol
lambda_ns_inv[mask] = eps_kswgd / (lambda_ns[mask] + reg)
phi_trunc = phi[:, :above_tol]
lambda_ns_s_ns = (lambda_ns_inv * inv_lambda * lambda_ns_inv)[:above_tol]

# Clean NaN and Inf
lambda_ns_s_ns = np.nan_to_num(lambda_ns_s_ns, nan=0.0, posinf=0.0, neginf=0.0)

# Target distribution weights
p_tar = np.sum(data_kernel, axis=0)
sqrt_p = np.sqrt(p_tar + 1e-12)
D_vec = np.sum(data_kernel / sqrt_p[:, None] / sqrt_p[None, :], axis=1)

print(f"\n✓ Eigendecomposition Summary:")
print(f"  Matrix size: {n_samples} x {n_samples}")
print(f"  Computed eigenvalues: {len(lambda_ns)} ({'truncated' if use_truncated else 'full'})")
print(f"  Eigenvalues >= {tol}: {above_tol}")
print(f"  Retained eigenvectors: {above_tol}")
print(f"  Top 10 eigenvalues: {lambda_ns[:10]}")
print(f"  lambda_ns_s_ns stats: min={lambda_ns_s_ns.min():.6f}, max={lambda_ns_s_ns.max():.6f}, non-zero={np.sum(lambda_ns_s_ns != 0)}")

### 6.2 EDMD Dictionary Learning for True Koopman Operator (KSWGD)

The previous cell computed the **Diffusion Map** spectral decomposition (DMPS method).
Now we implement the **true KSWGD** using EDMD (Extended Dynamic Mode Decomposition):

1. **KDE-based drift estimation**: Estimate the score function $\nabla \log p(x)$ using kernel density estimation
2. **Langevin evolution**: Generate time-series pairs $(X_t, X_{t+\Delta t})$ via Langevin dynamics
3. **Dictionary learning**: Learn a sparse dictionary on the latent space
4. **EDMD operator**: Build the Koopman operator in dictionary space

| Method | Basis Functions | Dynamics |
|--------|----------------|----------|
| DMPS | Diffusion map eigenvectors | Static kernel |
| KSWGD (EDMD) | Dictionary + Koopman eigenfunctions | Learned Langevin dynamics |

In [ ]:
# KDE-based drift estimation and Langevin evolution to build EDMD pairs
from sklearn.decomposition import MiniBatchDictionaryLearning
from scipy.linalg import eig

print("=== EDMD Pipeline for True Koopman KSWGD ===")

# Step 1: KDE-based score function estimation
dt_edmd = 0.1  # time step for EDMD Langevin evolution
dist2_edmd = pairwise_distances(X_tar, metric="sqeuclidean")  # squared distances
h_edmd = np.sqrt(np.median(dist2_edmd) + 1e-12)  # KDE bandwidth
W_edmd = np.exp(-dist2_edmd / (2.0 * (h_edmd ** 2)))
sumW_edmd = np.sum(W_edmd, axis=1, keepdims=True) + 1e-12
weighted_means_edmd = W_edmd @ X_tar / sumW_edmd
score_edmd = (weighted_means_edmd - X_tar) / (h_edmd ** 2)  # KDE score = drift term

# Step 2: Langevin step to generate (X_t, X_{t+dt}) pairs
xi_edmd = np.random.normal(0.0, 1.0, size=X_tar.shape)
X_tar_next = X_tar + dt_edmd * score_edmd + np.sqrt(2.0 * dt_edmd) * xi_edmd

print(f"EDMD drift bandwidth h: {h_edmd:.4f}")
print(f"X_tar stats -> mean {X_tar.mean():.4f}, std {X_tar.std():.4f}")
print(f"X_tar_next stats -> mean {X_tar_next.mean():.4f}, std {X_tar_next.std():.4f}")

In [ ]:
# Step 3: Dictionary learning on latent space
# Rule of thumb: dictionary size = 3-6x latent_dim, with N/m > 50 to avoid overfitting
# For N=30000, d=64: recommended m = 300-500 (gives N/m = 60-100)
n_dict_components = 200  # Dictionary atoms (total basis = n_dict_components + 1 for constant)
dict_alpha = 1e-3  # Sparsity penalty
dict_batch = 256
dict_max_iter = 500
dict_random_state = 42

print(f"\n=== Dictionary Learning ===")
print(f"Data: N={X_tar.shape[0]} samples, d={X_tar.shape[1]} dimensions")
print(f"Dictionary: {n_dict_components} atoms → {n_dict_components + 1} total basis functions")
print(f"Oversampling ratio: N/m = {X_tar.shape[0] / (n_dict_components + 1):.1f}x (recommended > 50)")
print(f"Learning dictionary...")

dict_model = MiniBatchDictionaryLearning(
    n_components=n_dict_components,
    alpha=dict_alpha,
    batch_size=dict_batch,
    max_iter=dict_max_iter,
    random_state=dict_random_state,
    verbose=0,
    fit_algorithm="lars"
)
dict_model.fit(X_tar)

# Transform to dictionary space and add constant term
Phi_X = dict_model.transform(X_tar)
Phi_Y = dict_model.transform(X_tar_next)
Phi_X = np.hstack([np.ones((Phi_X.shape[0], 1)), Phi_X])  # Add constant basis
Phi_Y = np.hstack([np.ones((Phi_Y.shape[0], 1)), Phi_Y])

print(f"Dictionary atoms shape: {dict_model.components_.shape}")
print(f"Dictionary codes (current) Phi_X: {Phi_X.shape}")
print(f"Dictionary codes (next) Phi_Y: {Phi_Y.shape}")

In [ ]:
# Step 4: Build EDMD Koopman operator in dictionary space
print("\n=== EDMD Koopman Operator ===")

reg_edmd = 1e-3
N_edmd, m_edmd = Phi_X.shape

# EDMD: K = (Phi_X^T Phi_X + reg*I)^{-1} Phi_X^T Phi_Y
G_edmd = (Phi_X.T @ Phi_X) / N_edmd + reg_edmd * np.eye(m_edmd)
A_edmd = (Phi_X.T @ Phi_Y) / N_edmd

# Generalized eigenvalue problem: A v = lambda G v
eigvals_edmd, eigvecs_edmd = eig(A_edmd, G_edmd)
idx_edmd = np.argsort(-eigvals_edmd.real)
eigvals_edmd = eigvals_edmd[idx_edmd]
eigvecs_edmd = eigvecs_edmd[:, idx_edmd]

# Koopman eigenfunctions evaluated at data points
efuns_edmd = Phi_X @ eigvecs_edmd

print(f"EDMD eigenvalues (first 10): {np.round(eigvals_edmd[:10].real, 4)}")
print(f"EDMD eigenfunctions shape: {efuns_edmd.shape}")

In [ ]:
# Step 5: Prepare EDMD-derived KSWGD weights (skip constant mode)
print("\n=== Prepare EDMD-KSWGD Spectral Weights ===")

lambda_ns_edmd = eigvals_edmd.real
lambda_gen_edmd = (lambda_ns_edmd - 1.0) / dt_edmd  # Generator eigenvalues

mode_skip_edmd = 1  # Skip the constant eigenfunction (eigenvalue ~ 1)
eig_threshold_edmd = 1e-6  # Keep modes with eigenvalue > threshold

valid_idx_edmd = np.arange(mode_skip_edmd, lambda_ns_edmd.shape[0])
valid_mask_edmd = lambda_ns_edmd[mode_skip_edmd:] > eig_threshold_edmd
valid_idx_edmd = valid_idx_edmd[valid_mask_edmd]

if valid_idx_edmd.size == 0:
    raise RuntimeError("No EDMD modes survived the threshold; adjust eig_threshold_edmd or dictionary size.")

# Truncated Koopman eigenfunctions for KSWGD
phi_trunc_edmd = np.real(efuns_edmd[:, valid_idx_edmd])

# Generator inverse weights
lambda_gen_inv_edmd = np.zeros_like(lambda_gen_edmd)
mask_nonzero_edmd = np.abs(lambda_gen_edmd) > 1e-6
lambda_gen_inv_edmd[mask_nonzero_edmd] = 1.0 / lambda_gen_edmd[mask_nonzero_edmd]
lambda_ns_s_ns_edmd = lambda_gen_inv_edmd[valid_idx_edmd].real

print(f"EDMD kept {valid_idx_edmd.size} Koopman modes (threshold {eig_threshold_edmd})")
print(f"phi_trunc_edmd shape: {phi_trunc_edmd.shape}")
print(f"lambda_ns_s_ns_edmd stats: min={lambda_ns_s_ns_edmd.min():.6f}, max={lambda_ns_s_ns_edmd.max():.6f}")

In [ ]:
# Compare Diffusion Map (DMPS) and EDMD (KSWGD) spectra
import matplotlib.pyplot as plt

n_show_eigs = 20
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Eigenvalue comparison
ax1 = axes[0]
ax1.semilogy(range(1, n_show_eigs + 1), lambda_ns[:n_show_eigs], 'o-', label="DMPS (Diffusion Map)", markersize=6)
ax1.semilogy(range(1, min(n_show_eigs, lambda_ns_edmd.size) + 1), 
             lambda_ns_edmd[:n_show_eigs], '^-', label="KSWGD (EDMD Koopman)", markersize=6)
ax1.set_xlabel("Eigen-index")
ax1.set_ylabel("Eigenvalue (log scale)")
ax1.set_title("Spectral Decay Comparison")
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Generator eigenvalues
ax2 = axes[1]
lambda_gen_dmps = (lambda_ns[:n_show_eigs] - 1.0) / eps_kswgd
ax2.plot(range(1, n_show_eigs + 1), lambda_gen_dmps, 'o-', label="DMPS Generator", markersize=6)
ax2.plot(range(1, min(n_show_eigs, lambda_gen_edmd.size) + 1), 
         lambda_gen_edmd[:n_show_eigs], '^-', label="EDMD Generator", markersize=6)
ax2.set_xlabel("Eigen-index")
ax2.set_ylabel("Generator Eigenvalue")
ax2.set_title("Generator Spectrum Comparison")
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.axhline(y=0, color='k', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig("figures/dmps_vs_kswgd_spectrum.png", dpi=150, bbox_inches='tight')
plt.show()

print("\n=== Method Comparison ===")
print(f"DMPS: {above_tol} diffusion map modes")
print(f"KSWGD: {valid_idx_edmd.size} Koopman modes from EDMD")

In [ ]:
# Define unified sampler supporting both DMPS and KSWGD (EDMD) methods
def run_particle_sampler(num_particles=16, num_iters=200, step_size=0.05, rng_seed=42, method="dmps"):
    """
    Unified Particle Transport Sampler
    
    Parameters:
    -----------
    method : str
        "dmps" - Diffusion Map Particle Sampling (uses diffusion map eigenvectors)
        "kswgd" - True KSWGD with EDMD Koopman operator
    """
    rng = np.random.default_rng(rng_seed)
    use_gpu = GPU_KSWGD and torch.cuda.is_available()
    xp = cp if use_gpu else np
    grad_fn = grad_ker1_gpu if use_gpu else grad_ker1
    K_eval_fn = K_tar_eval_gpu if use_gpu else K_tar_eval
    
    method = method.lower()
    if method == "dmps":
        method_name = "DMPS (Diffusion Map)"
        phi_use = phi_trunc
        lambda_use = lambda_ns_s_ns
    elif method == "kswgd":
        method_name = "KSWGD (EDMD Koopman)"
        phi_use = phi_trunc_edmd
        lambda_use = lambda_ns_s_ns_edmd
    else:
        raise ValueError(f"Unknown method: {method}. Use 'dmps' or 'kswgd'.")
    
    print(f"Method: {method_name}")
    print(f"Backend: {'GPU (CuPy)' if use_gpu else 'CPU (NumPy)'}")
    print(f"Eigenfunctions shape: {phi_use.shape}")
    
    x_hist = xp.zeros((num_particles, latent_dim, num_iters), dtype=xp.float64)
    init_particles = rng.normal(0.0, 1.0, size=(num_particles, latent_dim))
    x_hist[:, :, 0] = xp.asarray(init_particles)
    
    if use_gpu:
        X_tar_dev = cp.asarray(X_tar)
        p_tar_dev = cp.asarray(p_tar)
        sq_tar_dev = cp.asarray(sq_tar)
        D_vec_dev = cp.asarray(D_vec)
        phi_dev = cp.asarray(phi_use)
        lambda_dev = cp.asarray(lambda_use)
    else:
        X_tar_dev, p_tar_dev, sq_tar_dev, D_vec_dev = X_tar, p_tar, sq_tar, D_vec
        phi_dev, lambda_dev = phi_use, lambda_use
    
    iterator = trange(num_iters - 1, desc=f"{method.upper()} Transport", unit="step")
    for t in iterator:
        current = x_hist[:, :, t]
        grad_matrix = grad_fn(current, X_tar_dev, p_tar_dev, sq_tar_dev, D_vec_dev, eps_kswgd)
        cross_matrix = K_eval_fn(X_tar_dev, current, p_tar_dev, sq_tar_dev, D_vec_dev, eps_kswgd)
        
        tmp = (phi_dev.T @ cross_matrix) * lambda_dev[:, None]
        push = phi_dev @ tmp
        
        for dim in range(latent_dim):
            sum_term = grad_matrix[:, :, dim] @ push
            x_hist[:, dim, t + 1] = x_hist[:, dim, t] - (step_size / num_particles) * xp.sum(sum_term, axis=1)
        
        if (t + 1) % 50 == 0:
            step_norm = x_hist[:, :, t + 1] - x_hist[:, :, t]
            mean_disp = float(xp.mean(xp.linalg.norm(step_norm, axis=1)))
            iterator.set_postfix({"mean_step": f"{mean_disp:.3e}"})
            if bool(xp.any(xp.isnan(x_hist[:, :, t + 1]))):
                print(f"\nWarning: NaN detected at step {t+1}")
                return np.asarray(xp.asnumpy(x_hist[:, :, t]) if use_gpu else x_hist[:, :, t], dtype=np.float64)
    
    return np.asarray(xp.asnumpy(x_hist[:, :, -1]) if use_gpu else x_hist[:, :, -1], dtype=np.float64)


# Backward compatibility alias
def run_kswgd_sampler(num_particles=16, num_iters=200, step_size=0.05, rng_seed=42):
    """Legacy wrapper - uses DMPS by default for backward compatibility"""
    return run_particle_sampler(num_particles, num_iters, step_size, rng_seed, method="dmps")


def decode_latents_to_images(flat_latents_std):
    """Decode standardized latent vectors to images (supports both PCA and non-PCA modes)"""
    # 1. De-standardize
    flat_latents = flat_latents_std * Z_std + Z_mean
    
    # 2. PCA Inverse Transform (only if PCA was used)
    if USE_PCA and pca is not None:
        latents_recovered = pca.inverse_transform(flat_latents)
    else:
        latents_recovered = flat_latents  # Already in 1024-dim space
    
    # 3. Reshape to (N, 4, 16, 16)
    latents_tensor = torch.from_numpy(latents_recovered).float().view(-1, *full_latent_shape).to(device)
    
    # 4. VAE decode
    vae.eval()
    with torch.no_grad():
        decoded = vae.decode(latents_tensor / vae_scaling).sample
        decoded_rgb = _from_vae_range(decoded)
    
    return decoded_rgb.cpu()

print("Particle samplers and Decoder defined!")
print(f"  Latent mode: {'PCA (' + str(REDUCED_DIM) + '-dim)' if USE_PCA else 'Full latent (1024-dim)'}")
print(f"  Available methods:")
print(f"    - 'dmps': Diffusion Map Particle Sampling ({above_tol} modes)")
print(f"    - 'kswgd': EDMD Koopman KSWGD ({valid_idx_edmd.size} modes)")

### 7.1 Run Generation: DMPS vs KSWGD Comparison

Now we run both methods and compare results:
- **DMPS**: Diffusion Map Particle Sampling (faster, static kernel)
- **KSWGD**: EDMD-based Koopman Spectral Wasserstein Gradient Descent (learns dynamics)

In [ ]:
# ============== Spectral Memory Cleanup ==============
import gc
import sys
import torch
import psutil

print("Starting spectral memory cleanup after eigendecomposition...")

# 1. Record memory usage before cleanup
print("\n=== Memory Usage BEFORE Cleanup ===")
cpu_mem_before = psutil.virtual_memory().used / 1e9
gpu_mem_before = []
for i in range(torch.cuda.device_count()):
    gpu_allocated = torch.cuda.memory_allocated(i) / 1e9
    gpu_reserved = torch.cuda.memory_reserved(i) / 1e9
    gpu_mem_before.append((gpu_allocated, gpu_reserved))
    print(f"GPU {i}: Allocated: {gpu_allocated:.2f} GB, Reserved: {gpu_reserved:.2f} GB")
print(f"CPU RAM: {cpu_mem_before:.2f} GB")

# 2. Calculate and print sizes of variables to be deleted
print(f"\n=== Variables to be Released (max_samples = {max_samples}) ===")
memory_to_release = 0.0
variables_info = {}

# Variables that can be safely deleted after eigendecomposition
deletable_vars = [
    'dists',           # Distance matrix: (max_samples, max_samples) - huge!  
    'data_kernel',     # Original kernel matrix: (max_samples, max_samples)
    'rw_kernel',       # Random walk kernel: (max_samples, max_samples)  
    'phi',             # Full eigenvector matrix (before truncation)
    'lambda_ns',       # Full eigenvalue array (before truncation)
    'Z_flat',          # Original flattened latent vectors
    'all_latents',     # List of latent vectors
    '_eig_result',     # Temporary eigendecomposition results
]

for var in deletable_vars:
    if var in globals():
        obj = globals()[var]
        size_bytes = 0
        if hasattr(obj, 'nbytes'):
            size_bytes = obj.nbytes
        elif hasattr(obj, '__sizeof__'):
            size_bytes = obj.__sizeof__()
        elif isinstance(obj, (list, dict)):
            size_bytes = sys.getsizeof(obj)
            if isinstance(obj, list) and len(obj) > 0:
                # Estimate size for list of arrays
                if hasattr(obj[0], 'nbytes'):
                    size_bytes += sum(item.nbytes for item in obj if hasattr(item, 'nbytes'))
        
        size_gb = size_bytes / 1e9
        memory_to_release += size_gb
        variables_info[var] = size_gb
        print(f"  {var}: {size_gb:.3f} GB")

print(f"Total estimated memory to release: {memory_to_release:.2f} GB")

# 3. Ensure spectral info independence before deletion
print(f"\n=== Ensuring Spectral Data Independence ===")

# Ensure phi_trunc is independent from phi
if 'phi' in globals() and 'phi_trunc' in globals():
    if globals()['phi'] is globals()['phi_trunc']:
        print("⚠️  phi and phi_trunc are the same object - creating independent copy")
        globals()['phi_trunc'] = globals()['phi_trunc'].copy()
        print("✓ Made phi_trunc an independent copy.")
    else:
        print("✓ phi_trunc is already independent from phi")

# 4. Clear Python tracebacks (which can hold references)
sys.last_traceback = None
sys.last_value = None
sys.last_type = None

# 5. Perform the actual deletion
print(f"\n=== Deleting Large Matrices ===")
total_released = 0.0

for var in deletable_vars:
    if var in globals():
        size_gb = variables_info.get(var, 0)
        del globals()[var]
        total_released += size_gb
        print(f"✓ Deleted {var} ({size_gb:.3f} GB)")

# 6. Clean up any remaining CuPy arrays
if 'cp' in dir() and cp is not None:
    cp_vars_found = []
    for name in list(globals().keys()):
        if isinstance(globals().get(name), type(cp.ndarray(1))) if cp is not None else False:
            cp_vars_found.append(name)
            del globals()[name]
    if cp_vars_found:
        print(f"✓ Deleted CuPy arrays from globals: {', '.join(cp_vars_found)}")

# 7. Force memory release
print(f"\n=== Forcing Memory Release ===")
# Python garbage collection
for _ in range(3):
    n_collected = gc.collect()
    if n_collected > 0:
        print(f"✓ Python GC collected {n_collected} objects")

# CuPy memory pool cleanup
if 'cp' in dir() and cp is not None:
    try:
        mempool = cp.get_default_memory_pool()
        pinned_mempool = cp.get_default_pinned_memory_pool()
        
        for _ in range(3):
            mempool.free_all_blocks()
            pinned_mempool.free_all_blocks()
        
        print(f"✓ CuPy memory pool cleared and blocks freed.")
    except Exception as e:
        print(f"✗ CuPy cleanup error: {e}")

# PyTorch cleanup
torch.cuda.empty_cache()
print("✓ PyTorch CUDA cache emptied")

# 8. Record memory usage after cleanup  
print(f"\n=== Memory Usage AFTER Cleanup ===")
cpu_mem_after = psutil.virtual_memory().used / 1e9
gpu_mem_after = []
for i in range(torch.cuda.device_count()):
    gpu_allocated = torch.cuda.memory_allocated(i) / 1e9
    gpu_reserved = torch.cuda.memory_reserved(i) / 1e9
    gpu_mem_after.append((gpu_allocated, gpu_reserved))
    total = torch.cuda.get_device_properties(i).total_memory / 1e9
    print(f"GPU {i}: Allocated: {gpu_allocated:.2f} GB, Reserved: {gpu_reserved:.2f} GB, Free: {total - gpu_allocated:.2f} GB")
print(f"CPU RAM: {cpu_mem_after:.2f} GB")

# 9. Calculate and report memory savings
print(f"\n=== Memory Cleanup Summary ===")
cpu_released = cpu_mem_before - cpu_mem_after
print(f"CPU RAM released: {cpu_released:.2f} GB")

for i in range(len(gpu_mem_before)):
    gpu_released_allocated = gpu_mem_before[i][0] - gpu_mem_after[i][0]  
    gpu_released_reserved = gpu_mem_before[i][1] - gpu_mem_after[i][1]
    print(f"GPU {i} memory released: Allocated: {gpu_released_allocated:.2f} GB, Reserved: {gpu_released_reserved:.2f} GB")

print(f"Estimated variables released: {total_released:.2f} GB")

# 10. Verify essential spectral data is preserved
print(f"\n=== Essential Spectral Data Verification ===")
essential_vars = [
    'phi_trunc',        # Truncated eigenvectors (n_samples, above_tol)
    'lambda_ns_s_ns',   # Processed eigenvalue weights (above_tol,)
    'X_tar',            # Standardized target samples (n_samples, latent_dim)
    'p_tar',            # Target distribution weights (n_samples,)
    'sq_tar',           # Squared sums of target samples (n_samples,) 
    'D_vec',            # Normalization vector (n_samples,)
    'eps_kswgd',        # Kernel bandwidth parameter
    'Z_mean',           # Standardization mean
    'Z_std',            # Standardization std
]

all_essential_ok = True
for var in essential_vars:
    if var in globals():
        obj = globals()[var]
        if hasattr(obj, 'shape'):
            print(f"✓ {var}: shape {obj.shape}, dtype {obj.dtype}")
        else:
            print(f"✓ {var}: {type(obj).__name__} = {obj}")
    else:
        print(f"✗ {var}: MISSING!")
        all_essential_ok = False

if all_essential_ok:
    print(f"\n✅ All essential spectral data preserved! KSWGD sampling can proceed.")
else:
    print(f"\n❌ Some essential data is missing! Check before running KSWGD.")

print(f"\n✅ Spectral memory cleanup complete!")
print(f"   Total estimated release: {total_released:.2f} GB")
print(f"   Kernel matrix ({max_samples}x{max_samples}) and full eigenvectors deleted")
print(f"   Truncated spectral info (k_eig={k_eig}) preserved for KSWGD")

In [ ]:
# Run KSWGD sampling to generate new latent vectors
kswgd_config = {
    "num_particles": n_generate,  # (n_generate) Same as LDM generation count for fair comparison
    "num_iters": 500,             # Number of iterations
    "step_size": 0.05,            # Step size (smaller for stability)
    "rng_seed": 42
}

print("=" * 50)
print("KSWGD Generation Config:")
for k, v in kswgd_config.items():
    print(f"  {k}: {v}")
print("=" * 50)
print(f"\nNote: Generating {kswgd_config['num_particles']} particles for reliable FID...")
print("This will take a while...")

# Run KSWGD (progress bar is built into run_kswgd_sampler via trange)
start_time = time.time()
Z_kswgd_std = run_kswgd_sampler(**kswgd_config)
kswgd_time = time.time() - start_time

print(f"\n✓ KSWGD Complete!")
print(f"Generated samples shape: {Z_kswgd_std.shape}")
print(f"Total time: {kswgd_time:.1f} seconds ({kswgd_time/60:.1f} minutes)")

In [ ]:
# Decode KSWGD generated latent vectors to images (in batches to avoid OOM)
print(f"Decoding {Z_kswgd_std.shape[0]} latent vectors to images...")

decode_batch_size = 128  # Increased for 80GB GPUs
all_kswgd_images = []

for i in tqdm(range(0, Z_kswgd_std.shape[0], decode_batch_size), desc="Decoding KSWGD"):
    batch_latents = Z_kswgd_std[i:i+decode_batch_size]
    batch_images = decode_latents_to_images(batch_latents)
    all_kswgd_images.append(batch_images.numpy())
    
    # Clear cache periodically
    if (i // decode_batch_size + 1) % 20 == 0:
        torch.cuda.empty_cache()

kswgd_images_np = np.concatenate(all_kswgd_images, axis=0)

print(f"\n✓ Decoding complete!")
print(f"Generated images shape: {kswgd_images_np.shape}")
print(f"Pixel value range: [{kswgd_images_np.min():.3f}, {kswgd_images_np.max():.3f}]")

In [ ]:
# Visualize KSWGD generated images (show first 16 of 10000)
n_show = min(16, kswgd_images_np.shape[0])
n_cols = 4
n_rows = (n_show + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(3 * n_cols, 3 * n_rows))
axes = np.asarray(axes).reshape(-1)

for idx in range(n_show):
    img = np.transpose(kswgd_images_np[idx], (1, 2, 0))  # (C,H,W) → (H,W,C)
    axes[idx].imshow(np.clip(img, 0.0, 1.0))
    axes[idx].set_title(f"KSWGD #{idx+1}")
    axes[idx].axis('off')

for idx in range(n_show, len(axes)):
    axes[idx].axis('off')

# Get image size from KSWGD generated images (shape is N, C, H, W)
_kswgd_img_size = kswgd_images_np.shape[2]
plt.suptitle(f"KSWGD CelebA-HQ ({_kswgd_img_size}x{_kswgd_img_size})", fontsize=22)
plt.tight_layout()
plt.show()

print(f"\nGeneration Summary:")
print(f"  - LDM: {len(ldm_generated_images)} images via UNet Denoising (200 steps)")
print(f"  - KSWGD: {kswgd_images_np.shape[0]} images via KSWGD ({kswgd_config['num_iters']} steps)")

In [ ]:
# ============== Apply GFPGAN to KSWGD Generated Images ==============
print("=" * 60)
print("Upscaling KSWGD Generated Images with GFPGAN")
print("(with preprocessing: Gaussian + Bilateral + Color Norm)")
print("=" * 60)

# Convert KSWGD images to the right format for upscaling
kswgd_images_for_upscale = []
for i in range(kswgd_images_np.shape[0]):
    img = np.transpose(kswgd_images_np[i], (1, 2, 0))  # (C,H,W) → (H,W,C)
    img = np.clip(img, 0, 1)
    kswgd_images_for_upscale.append(img)

print(f"\nEnhancing {len(kswgd_images_for_upscale)} KSWGD images with preprocessing + GFPGAN...")
kswgd_enhanced = upscale_images(kswgd_images_for_upscale, use_face_enhance=True, use_preprocess=True, desc="Preprocess+GFPGAN (KSWGD)")

# Visualization (show up to 16 in a grid)
n_test_kswgd = len(kswgd_enhanced)
n_show_kswgd = min(16, n_test_kswgd)
n_cols = 4
n_rows = (n_show_kswgd + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(4 * n_cols, 4 * n_rows))
axes = np.asarray(axes).reshape(-1)

for i in range(n_show_kswgd):
    axes[i].imshow(kswgd_enhanced[i])
    # axes[i].set_title(f"#{i+1}")
    axes[i].axis('off')

for i in range(n_show_kswgd, len(axes)):
    axes[i].axis('off')

# Get image size from enhanced images
_kswgd_enhanced_size = kswgd_enhanced[0].shape[1] if isinstance(kswgd_enhanced[0], np.ndarray) else kswgd_enhanced[0].size[0]
plt.suptitle(f"KSWGD CelebA-HQ ({_kswgd_enhanced_size}x{_kswgd_enhanced_size})\n", fontsize=22, y=1.02)
plt.tight_layout()
plt.savefig('/workspace/kswgd/figures/kswgd_gfpgan_enhanced.png', dpi=150, bbox_inches='tight')
plt.show()

# Store enhanced KSWGD images
kswgd_upscaled = kswgd_enhanced

print(f"\n✓ KSWGD Enhancement complete!")
print(f"  Original KSWGD: 256x256")
print(f"  Enhanced: {kswgd_upscaled[0].shape[1]}x{kswgd_upscaled[0].shape[0]}")
print(f"  Preprocessing applied: Gaussian blur (3x3, σ=0.5) + Bilateral (d=5) + Color norm (α=0.3)")

### 7.2 FID (Fréchet Inception Distance) Evaluation

Compute FID scores to quantitatively compare LDM and KSWGD generation quality.

**FID** measures the distance between the distribution of generated images and real images in Inception-v3 feature space. **Lower is better.**

In [ ]:
# ============== FID Utility Functions (Dual-GPU Support) ==============
# Install pytorch-fid if needed
import subprocess
import sys
from concurrent.futures import ThreadPoolExecutor

try:
    from pytorch_fid import fid_score
    from pytorch_fid.inception import InceptionV3
except ImportError:
    print("Installing pytorch-fid...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pytorch-fid", "-q"])
    from pytorch_fid import fid_score
    from pytorch_fid.inception import InceptionV3

from scipy import linalg
from torchvision import transforms as T

# Check number of GPUs for FID computation
NUM_GPUS_FID = torch.cuda.device_count()
USE_DUAL_GPU_FID = NUM_GPUS_FID >= 2
print(f"FID computation: Detected {NUM_GPUS_FID} GPU(s), using {'Dual-GPU' if USE_DUAL_GPU_FID else 'Single-GPU'} mode")

def get_inception_features_single_gpu(images, gpu_id, batch_size=256, desc="Extracting features"):
    """Extract Inception-v3 features on a single GPU"""
    device_local = torch.device(f"cuda:{gpu_id}")
    
    block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
    inception = InceptionV3([block_idx]).to(device_local)
    inception.eval()
    
    preprocess = T.Compose([
        T.Resize((299, 299)),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    features_list = []
    n_batches = (len(images) + batch_size - 1) // batch_size
    
    with torch.no_grad():
        for i in tqdm(range(0, len(images), batch_size), total=n_batches, desc=f"{desc} (GPU {gpu_id})"):
            batch_samples = images[i:i+batch_size]
            
            tensors = []
            for img in batch_samples:
                if isinstance(img, Image.Image):
                    t = T.ToTensor()(img)
                elif isinstance(img, np.ndarray):
                    t = torch.from_numpy(img).float()
                    if t.ndim == 3:
                        if t.shape[2] == 3 and t.shape[0] != 3:
                            t = t.permute(2, 0, 1)
                    if t.max() > 1.0: t /= 255.0
                else:
                    t = torch.as_tensor(img).float()
                tensors.append(t)
            
            batch = torch.stack(tensors).to(device_local)
            batch = preprocess(batch)
            feat = inception(batch)[0]
            feat = feat.squeeze(-1).squeeze(-1)
            features_list.append(feat.cpu().numpy())
    
    del inception
    torch.cuda.empty_cache()
    
    return np.concatenate(features_list, axis=0)

def get_inception_features(images, batch_size=256, desc="Extracting features"):
    """
    Extract Inception-v3 features from images.
    Automatically uses dual-GPU parallel processing if available.
    """
    n_images = len(images)
    
    if USE_DUAL_GPU_FID and n_images >= 64:  # Only use dual-GPU for larger batches
        print(f"  Using Dual-GPU parallel feature extraction...")
        mid = n_images // 2
        
        images_gpu0 = images[:mid] if isinstance(images, list) else list(images[:mid])
        images_gpu1 = images[mid:] if isinstance(images, list) else list(images[mid:])
        
        # Use ThreadPoolExecutor to run both GPUs in parallel
        with ThreadPoolExecutor(max_workers=2) as executor:
            future_gpu0 = executor.submit(
                get_inception_features_single_gpu, images_gpu0, 0, batch_size, desc
            )
            future_gpu1 = executor.submit(
                get_inception_features_single_gpu, images_gpu1, 1, batch_size, desc
            )
            
            features_gpu0 = future_gpu0.result()
            features_gpu1 = future_gpu1.result()
        
        return np.concatenate([features_gpu0, features_gpu1], axis=0)
    else:
        # Single-GPU processing
        return get_inception_features_single_gpu(images, 0, batch_size, desc)

def calculate_fid(real_features, gen_features):
    """Calculate FID between two sets of features"""
    mu_real = np.mean(real_features, axis=0)
    mu_gen = np.mean(gen_features, axis=0)
    sigma_real = np.cov(real_features, rowvar=False)
    sigma_gen = np.cov(gen_features, rowvar=False)
    
    # Calculate FID
    diff = mu_real - mu_gen
    # Add a small value to diagonal for stability
    offset = np.eye(sigma_real.shape[0]) * 1e-6
    covmean, _ = linalg.sqrtm((sigma_real + offset) @ (sigma_gen + offset), disp=False)
    
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    
    fid = diff @ diff + np.trace(sigma_real + sigma_gen - 2 * covmean)
    return float(fid)

print("✓ FID utility functions defined (with Dual-GPU support).")

In [ ]:
# ============== FID Evaluation (Raw 256x256 Images) ==============
print("=" * 60)
print("Computing FID Scores (on Raw 256x256 Images)...")
print(f"  LDM raw:   {len(ldm_generated_images)} images")
print(f"  KSWGD raw: {len(kswgd_images_np)} images")
print("=" * 60)

# 1. Get real images from CelebA-HQ (resized to 256x256)
n_real_samples = min(10000, len(celebahq_dataset))  # 10k real images for reliable FID
real_images_256 = []
for i in tqdm(range(n_real_samples), desc="Loading real images (256)"):
    img = celebahq_dataset[i]["image"].resize((256, 256))
    real_images_256.append(np.array(img))

print(f"  Real images loaded: {len(real_images_256)}")

# 2. Extract features
print("\n[1/3] Extracting Inception features from real images (256)...")
real_features_256 = get_inception_features(real_images_256, desc="Real images (256)")

print("\n[2/3] Extracting Inception features from LDM raw images (256)...")
ldm_features_256 = get_inception_features(ldm_generated_images, desc="LDM Raw (256)")

print("\n[3/3] Extracting Inception features from KSWGD raw images (256)...")
kswgd_features_256 = get_inception_features(kswgd_images_np, desc="KSWGD Raw (256)")

# 3. Calculate FID
print("\nCalculating FID scores...")
fid_ldm_raw = calculate_fid(real_features_256, ldm_features_256)
fid_kswgd_raw = calculate_fid(real_features_256, kswgd_features_256)

print("\n" + "=" * 60)
print("FID RESULTS (Raw 256x256, Lower is Better)")
print("=" * 60)
print(f"  Real images:  {n_real_samples}")
print(f"  LDM (Raw):   {len(ldm_generated_images)} → FID = {fid_ldm_raw:.2f}")
print(f"  KSWGD (Raw): {len(kswgd_images_np)} → FID = {fid_kswgd_raw:.2f}")
print("=" * 60)
print(f"\nNote: This FID measures the quality of the raw 256x256 generation before any enhancement.")

In [ ]:
# ============== FID Evaluation (Enhanced 1024x1024 Images) ==============
print("=" * 60)
print("Computing FID Scores (on 1024x1024 Enhanced Images)...")
print(f"  LDM enhanced:   {len(ldm_upscaled)} images")
print(f"  KSWGD enhanced: {len(kswgd_upscaled)} images")
print("=" * 60)

# 1. Get real images from CelebA-HQ (1024x1024)
print("\n[1/4] Loading real CelebA-HQ images (1024x1024)...")
# Use a smaller number for real images if memory is tight, but 1000-2000 is usually enough for comparison
n_real_samples = min(10000, len(celebahq_dataset))  # 10k real images for reliable FID
real_images_1024 = []
for i in tqdm(range(n_real_samples), desc="Loading real images (1024)"):
    img = celebahq_dataset[i]["image"]
    # No resize, keep original 1024x1024
    img_tensor = T.ToTensor()(img) 
    real_images_1024.append(img_tensor.numpy())

print(f"  Real images loaded: {len(real_images_1024)}")

print("\n[2/4] Extracting Inception features from real images (1024)...")
real_features_1024 = get_inception_features(real_images_1024, desc="Real images (1024)")

# 2. Get LDM enhanced images features
print("\n[3/4] Extracting Inception features from LDM enhanced images...")
ldm_features_1024 = get_inception_features(ldm_upscaled, desc="LDM Enhanced")

# 3. Get KSWGD enhanced images features  
print("\n[4/4] Extracting Inception features from KSWGD enhanced images...")
kswgd_features_1024 = get_inception_features(kswgd_upscaled, desc="KSWGD Enhanced")

# 4. Calculate FID scores
print("\nCalculating FID scores...")
fid_ldm_enhanced = calculate_fid(real_features_1024, ldm_features_1024)
fid_kswgd_enhanced = calculate_fid(real_features_1024, kswgd_features_1024)

print("\n" + "=" * 60)
print("FID RESULTS (Enhanced 1024x1024, Lower is Better)")
print("=" * 60)
print(f"  Real images:  {n_real_samples}")
print(f"  LDM (Enhanced):   {len(ldm_upscaled)} → FID = {fid_ldm_enhanced:.2f}")
print(f"  KSWGD (Enhanced): {len(kswgd_upscaled)} → FID = {fid_kswgd_enhanced:.2f}")
print("=" * 60)
print(f"\nNote: This FID measures the quality of the full pipeline (Model + Preprocessing + GFPGAN).")

## 9. Clean Up GPU Memory

In [ ]:
# Clean up GPU memory
import gc

if 'ldm_pipe' in dir():
    del ldm_pipe
if 'vae' in dir():
    del vae
gc.collect()
torch.cuda.empty_cache()

print(f"Current GPU memory usage: {torch.cuda.memory_allocated() / 1e9:.2f} GB")
print(f"GPU memory cached: {torch.cuda.memory_reserved() / 1e9:.2f} GB")